# 1.Data Collection

In [1]:
import os
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import pandas as pd
from nltk.stem import *
import time
import nltk
from nltk.corpus import stopwords
from collections import Counter
import random
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import string
import re
from forex_python.converter import CurrencyRates
from collections import defaultdict
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer


In [2]:
url = 'https://www.findamasters.com/masters-degrees/msc-degrees/'
result = requests.get(url)
soup = BeautifulSoup(result.text)
lst_links = soup.find_all('a', {'class': 'courseLink'})

In [3]:
def extract_masters(this_url):
  result_url = requests.get(this_url)
  result_soup = BeautifulSoup(result_url.text)
  result_links = result_soup.find_all('a', {'class': 'courseLink'})
  result_list = []
  for item in result_links:
    result_list.append((item['href'], item.text))

  return result_list

Extract Masters and create a txt 

In [ ]:
'''
num_pages = 400
pref = 'https://www.findamasters.com'
test_lst_all = []

for i in range(1, num_pages + 1):
    test_lst = extract_masters(pref + '/masters-degrees/msc-degrees/?PG=' + str(i))
    test_lst_all.extend(test_lst)
with open('masters_urls.txt', 'w') as f:
    for item in test_lst_all:
        f.write(item[0] + '\n')

'''

Save HTML file

In [4]:
def extract_msc_page(msc_page_url):
    contents = {'url': msc_page_url}
    try:
        page_url = requests.get(msc_page_url)
        page_soup = BeautifulSoup(page_url.text, 'html.parser')
        #Course Name
        name_links = page_soup.find_all('h1', {'class':'course-header__course-title'})
        if name_links:
            contents['courseName'] = name_links[0].text.strip()
        else:
            contents['courseName'] = ''       
        # University name
        uni_links = page_soup.find_all('a', {'class': 'course-header__institution'})
        if uni_links:
            contents['universityName'] = uni_links[0].get_text(strip=True)
        else:
            contents['universityName'] = ''

        # Faculty Name
        faculty_links = page_soup.find_all('a', {'class': 'course-header__department'})
        if faculty_links:
            contents['facultyName'] = faculty_links[0].get_text(strip=True)
        else:
            contents['facultyName'] = ''

        # Full time or not
        FullTime_links = page_soup.find_all('a', {'class':'concealLink' })
        FullTime = any(item['href'] == "/masters-degrees/full-time/" for item in FullTime_links)
        contents['fullTime'] = FullTime

        # Description
        description_element = page_soup.find("div", id='Snippet')
        if description_element:
            contents['description'] = description_element.get_text(strip=True)
        else:
            contents['description'] = ''

        # Start Date
        start_links = page_soup.find('span', {'class': 'key-info__start-date'})
        if start_links:
            contents['startDate'] = start_links.get_text(strip=True)
        else:
            contents['startDate'] = ''

        # Fees
        fees_element = page_soup.find('div', {'class': "course-sections__fees"})
        if fees_element:
            fees_links = fees_element.get_text(strip=True)
            contents['fees'] = fees_links
        else:
            contents['fees'] = ''

        # Modality
        modality_element = page_soup.find('span', {'class': "key-info__qualification"})
        if modality_element:
            modality_links = modality_element.get_text(strip=True)
            contents['modality'] = modality_links
        else:
            contents['modality'] = ''

        # Duration
        duration_links = page_soup.find('span', {'class': "key-info__duration"})
        if duration_links:
            contents['duration'] = duration_links.get_text(strip=True)
        else:
            contents['duration'] = ''

        # City
        city_links = page_soup.find_all('a', {'class':'course-data__city'})
        if city_links:
            contents['city'] = city_links[0].get_text(strip=True)
        else:
            contents['city'] = ''

        # Country
        country_links = page_soup.find_all('a', {'class': 'course-data__country'})
        if country_links:
            contents['country'] = country_links[0].get_text(strip=True)
        else:
            contents['country'] = ''

        # Administrator
        admin_links = page_soup.find_all('a', {'class': 'course-data__on-campus'})
        if admin_links:
            contents['administator'] = admin_links[0].get_text(strip=True)
        else:
            contents['administator'] = ''

    except Exception as e:
        print(f"Error processing {msc_page_url}: {e}")

    return contents


In [5]:
# Impostazioni per l'User-Agent per simulare un browser
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36'}
folder_name = "html-pages"
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# Initialize an empty list to store extracted data
data_list = []

# Your loop for creating folders
for i in range(1, 401):
    name = 'HTML page ' + str(i)
    path_folder = os.path.join(folder_name, name)
    if not os.path.exists(path_folder):
        os.makedirs(path_folder)

# Apre il file contenente gli URL
with open('masters_urls.txt', 'r') as file:
    for index, url in enumerate(file):
        url = url.strip()
        page = (index // 15) + 1
        try:
            full_url = "https://www.findamasters.com" + url.strip()
            time.sleep(1 + random.uniform(0, 4))
            response = requests.get(full_url, headers=headers)

            if response.status_code == 200:
                soup = BeautifulSoup(response.content, 'html.parser')
                file_path = os.path.join(f"{folder_name}{os.path.sep}HTML page {page}", f"course {index+1}.html")
                with open(file_path, "w", encoding="utf-8") as html_file:
                    html_file.write(str(soup))

                # Extract data and append to the list
                data_list.append(extract_msc_page(full_url))

            else:
                print(f"Error retrieving the course page:{full_url}")

        except Exception as e:
            print(f"Error during retrieval and saving of the page {full_url}: {e}")

# Convert the list of dictionaries to a Pandas DataFrame
df = pd.DataFrame(data_list)

# Save the DataFrame to a CSV file
df.to_csv("masters_dataset.csv", index=False)


Error retrieving the course page:https://www.findamasters.com/masters-degrees/course/business-analytics-part-time-online-msc/?i112d3962c72183
Error retrieving the course page:https://www.findamasters.com/masters-degrees/course/business-analytics-online-msc/?i280d8352c59504
Error retrieving the course page:https://www.findamasters.com/masters-degrees/course/business-and-management-msc-online/?i180d8375c66140
Error retrieving the course page:https://www.findamasters.com/masters-degrees/course/business-in-technology-msc/?i364d8644c68504
Error retrieving the course page:https://www.findamasters.com/masters-degrees/course/cardiovascular-research-msc/?i132d4846c17710
Error retrieving the course page:https://www.findamasters.com/masters-degrees/course/cardiovascular-sciences-msc-medsci/?i307d4814c14387
Error retrieving the course page:https://www.findamasters.com/masters-degrees/course/care-of-the-older-person-msc/?i1705d6873c57069
Error retrieving the course page:https://www.findamasters.com

In [6]:
df = pd.read_csv(r"C:\Users\Rebecca\Desktop\Magistrale\ADM\Adm_hw3\masters_dataset.csv")
df.head()

,url,courseName,universityName,facultyName,fullTime,description,startDate,fees,modality,duration,city,country,administator
0,https://www.findamasters.com/masters-degrees/c...,3D Design for Virtual Environments - MSc,Glasgow Caledonian University,School of Engineering and Built Environment,True,3D visualisation and animation play a role in ...,September,FeesPlease see the university website for furt...,MSc,1 year full-time,Glasgow,United Kingdom,On Campus
1,https://www.findamasters.com/masters-degrees/c...,Accounting and Finance - MSc,University of Leeds,Leeds University Business School,True,Businesses and governments rely on sound finan...,September,"FeesUK: £18,000 (Total)International: £34,750 ...",MSc,1 year full time,Leeds,United Kingdom,On Campus
2,https://www.findamasters.com/masters-degrees/c...,"Accounting, Accountability & Financial Managem...",King’s College London,King’s Business School,True,"Our Accounting, Accountability & Financial Man...",September,FeesPlease see the university website for furt...,MSc,1 year FT,London,United Kingdom,On Campus
3,https://www.findamasters.com/masters-degrees/c...,"Accounting, Financial Management and Digital B...",University of Reading,Henley Business School,True,Embark on a professional accounting career wit...,September,FeesPlease see the university website for furt...,MSc,1 year full time,Reading,United Kingdom,On Campus
4,https://www.findamasters.com/masters-degrees/c...,Addictions MSc,King’s College London,"Institute of Psychiatry, Psychology and Neuros...",True,Join us for an online session for prospective ...,September,FeesPlease see the university website for furt...,MSc,One year FT,London,United Kingdom,On Campus


Create tsv files

In [7]:
# Save each row as a separate TSV file
output_folder = 'course_data_tsv'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

for index, row in df.iterrows():
    course_name = f"course_{index + 1}.tsv"
    output_path = os.path.join(output_folder, course_name)
    row.to_csv(output_path, sep='\t', index=False)


# 2.Search Engine

Preprocessing

In [8]:
# Download NLTK resources if not already downloaded
nltk.download('stopwords')
nltk.download('punkt')

# Define or import your list of stopwords and punctuation
stop_words = set(stopwords.words('english'))
punctuation = set(string.punctuation)

# Initialize stemmer
stemmer = PorterStemmer()

# Function to preprocess text
def preprocess_text(text):
    if pd.isnull(text):
        return ""  # Return an empty string for missing values
    tokens = word_tokenize(str(text).lower())  # Ensure text is converted to string before processing
    processed_tokens = [word for word in tokens if word.isalnum() and word not in stop_words and word not in punctuation]
    return ' '.join(processed_tokens)

# Assuming df is your DataFrame and 'description' is the column to preprocess
df['preprocessed_desc'] = df['description'].apply(preprocess_text)

# Function to perform stemming on a sentence
def stem_sentence(sentence):
    return [stemmer.stem(word) for word in word_tokenize(sentence)]

# Apply stemming to 'preprocessed_desc' column
df['descr_stem'] = df['preprocessed_desc'].apply(stem_sentence)

# Display or use the DataFrame with the 'descr_stem' column containing lists of stemmed words
print(df['descr_stem'])


[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading punkt: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


0       [3d, visualis, anim, play, role, mani, area, p...
1       [busi, govern, reli, sound, financi, knowledg,...
2       [account, account, financi, manag, msc, cours,...
3       [embark, profession, account, career, academ, ...
4       [join, us, onlin, session, prospect, student, ...
                              ...                        
5300    [numer, method, knowledg, relationship, manufa...
5301    [regist, interest, graduat, studi, uclther, gr...
5302    [provid, uniqu, master, educ, materi, chemistr...
5303    [programm, descriptionmateri, chemistri, emerg...
5304    [themast, degre, materi, engineeringi, interdi...
Name: descr_stem, Length: 5305, dtype: object


Preprocessing the fees column

In [9]:
# Function to extract numerical fee values
def extract_numerical_fee(text):
    numerical_values = re.findall(r'\d+\.?\d*', str(text))  # Ensure text is converted to string
    return max(map(float, numerical_values)) if numerical_values else None  # Return the maximum value

# Clean textual fee data and extract numerical values
df['numeric_fee'] = df['fees'].apply(lambda x: extract_numerical_fee(x) if isinstance(x, str) else None)

# Ensure the 'currency_code' column exists and contains valid currency codes
df['currency_code'] = 'USD'  # Replace 'USD' with the correct currency code column

def convert_to_common_currency(row):
    if pd.notnull(row['numeric_fee']) and pd.notnull(row['currency_code']):
        c = CurrencyRates()
        conversion_rate = c.get_rate(row['currency_code'], 'USD')
        return row['numeric_fee'] * conversion_rate
    else:
        return None

df['fees_usd'] = df.apply(convert_to_common_currency, axis=1)



In [10]:
print(df['fees_usd'] )

0          NaN
1        750.0
2          NaN
3          NaN
4          NaN
         ...  
5300       NaN
5301     100.0
5302       NaN
5303       NaN
5304    2700.0
Name: fees_usd, Length: 5305, dtype: float64


Conjunctive query\
For the first version of the search engine, we narrowed our interest to the description of each course. It means that you will evaluate queries only concerning the course's description.

In [17]:
df['text'] = df['descr_stem'].apply(lambda x: ' '.join(x))

# Create an empty vocabulary and inverted index
vocabulary = {}
inverted_index = defaultdict(list)
term_id = 1  # Start term_id from 1

def tokenize(text):
    if pd.isnull(text):
        return []
    # Split the preprocessed and stemmed text into words
    return text.split()

# Loop through each row in the DataFrame
for index, row in df.iterrows():
    words = tokenize(row['text'])
    
    # Keep track of unique words in each document to avoid duplicate indexing
    unique_words_in_document = set()
    
    # Loop through each word in the text
    for word in words:
        if word not in vocabulary:
            vocabulary[word] = term_id
            term_id += 1
        
        term_id_for_word = vocabulary[word]

        # Ensure the document is added only once for each term
        if index not in inverted_index[term_id_for_word]:
            inverted_index[term_id_for_word].append(index)
            
            # Add the word to the set of unique words in this document
            unique_words_in_document.add(term_id_for_word)



print("\nVocabulary:")
print(vocabulary)
print("\nInverted Index:")
print(inverted_index)


Vocabulary:
{'3d': 1, 'visualis': 2, 'anim': 3, 'play': 4, 'role': 5, 'mani': 6, 'area': 7, 'popular': 8, 'media': 9, 'keep': 10, 'grow': 11, 'digit': 12, 'provid': 13, 'special': 14, 'effect': 15, '21st': 16, 'centuri': 17, 'favourit': 18, 'film': 19, 'televis': 20, 'show': 21, 'design': 22, 'also': 23, 'essenti': 24, 'everyday': 25, 'work': 26, 'everyth': 27, 'comput': 28, 'game': 29, 'develop': 30, 'onlin': 31, 'virtual': 32, 'world': 33, 'industri': 34, 'market': 35, 'product': 36, 'programm': 37, 'environ': 38, 'help': 39, 'skill': 40, 'thrive': 41, 'success': 42, 'career': 43, 'visual': 44, 'practic': 45, 'orient': 46, 'toward': 47, 'current': 48, 'need': 49, 'technolog': 50, 'prior': 51, 'knowledg': 52, 'requir': 53, 'busi': 54, 'govern': 55, 'reli': 56, 'sound': 57, 'financi': 58, 'underpin': 59, 'strategi': 60, 'cours': 61, 'profession': 62, 'advanc': 63, 'modern': 64, 'theori': 65, 'account': 66, 'control': 67, 'well': 68, 'understand': 69, 'organis': 70, 'cover': 71, 'funda

Execute the query

In [18]:
def search_documents(query):
    # Preprocess the query similarly to how you processed the descriptions
    query_tokens = word_tokenize(preprocess_text(query))
    query_tokens = [stemmer.stem(word) for word in query_tokens if word not in stop_words and word not in punctuation]

    # Initialize a list to store information for the relevant documents
    relevant_docs_info = []

    # Loop through each row in the DataFrame
    for index, row in df.iterrows():
        # Get the stemmed tokens from the 'descr_stem' column for the current document
        doc_tokens = set(row['descr_stem'])

        # Check if all query tokens are present in the current document
        if all(token in doc_tokens for token in query_tokens):
            relevant_doc_info = {
                'courseName': row['courseName'],
                'universityName': row['universityName'],
                'description': row['description'],
                'url': row['url']
            }
            relevant_docs_info.append(relevant_doc_info)

    # Create a new DataFrame from the collected information
    relevant_docs_df = pd.DataFrame(relevant_docs_info)

    return relevant_docs_df





In [19]:
# Execute a query
user_query = "Business"
relevant_documents = search_documents(user_query)
relevant_documents.head()



,courseName,universityName,description,url
0,Accounting and Finance - MSc,University of Leeds,Businesses and governments rely on sound finan...,https://www.findamasters.com/masters-degrees/c...
1,"Accounting, Financial Management and Digital B...",University of Reading,Embark on a professional accounting career wit...,https://www.findamasters.com/masters-degrees/c...
2,Advanced Master in Innovation & Strategic Mana...,Solvay Brussels School,Programme overviewThe Advanced Master in Innov...,https://www.findamasters.com/masters-degrees/c...
3,"Agricultural, Environmental and Resource Econo...",University of Helsinki,Goal of the pro­grammeAre you looking forward ...,https://www.findamasters.com/masters-degrees/c...
4,Applied Statistics (online) MSc,University of Strathclyde,Ouronline MSc in Applied Statisticsis a conver...,https://www.findamasters.com/masters-degrees/c...


For the second search engine, given a query, we want to get the top-k (the choice of k it's up to you!) documents related to the query. In particular:

- Find all the documents that contain all the words in the query.
- Sort them by their similarity with the query.
- Return in output k documents, or all the documents with non-zero similarity with the query when the results are less than k. You must use a heap data structure (you can use  Python libraries) for maintaining the top-k documents.

To solve this task, you must use the tfIdf score and the Cosine similarity. The field to consider is still the description. Let's see how.

In [13]:

tfidf_vectorizer = TfidfVectorizer()

# Fit and transform the data to compute TF-IDF scores
tfidf_matrix = tfidf_vectorizer.fit_transform(df['text'])

# Get the feature names (terms) from the vectorizer
terms = tfidf_vectorizer.get_feature_names()

# Create an inverted index
inverted_index_tfidf = {}

# Loop through each term's index and documents to build the inverted index
for term_index, term in enumerate(terms):
    inverted_index_tfidf[term] = []
    for document_index in range(len(df)):  # Loop through document indices directly
        tfidf_score = tfidf_matrix[document_index, term_index]
        if tfidf_score > 0:
            # Use the document index as the identifier
            inverted_index_tfidf[term].append((df.index[document_index], tfidf_score))




In [14]:
print(inverted_index_tfidf['aim'])

[(27, 0.07208949092678471), (33, 0.05658503485848769), (63, 0.12466950401056), (65, 0.1045107122490147), (75, 0.10475124441000583), (87, 0.0829229452702873), (136, 0.09964606339967527), (147, 0.07702046511710985), (176, 0.14916359335607185), (185, 0.10569453943729117), (199, 0.1010472295105067), (215, 0.08517052858917383), (255, 0.10165424169394774), (256, 0.10359284255900629), (260, 0.0858163103293614), (278, 0.08255231123451982), (279, 0.09256256884359264), (281, 0.09395464612210436), (284, 0.09157832897454367), (286, 0.06765037130669266), (293, 0.11329008856866839), (295, 0.16269373930757788), (329, 0.12034554049845261), (336, 0.09510759454654429), (388, 0.12025912304272184), (404, 0.07302670048493387), (453, 0.07833253896054908), (454, 0.13176484900554675), (455, 0.0953021840574058), (456, 0.07556288476235629), (463, 0.09401114714379637), (470, 0.12329293912991099), (476, 0.06312116966272403), (492, 0.07856134414453628), (494, 0.12008299469987194), (505, 0.18058837453872534), (508,

In [15]:

query = "aim"
query_tfidf = tfidf_vectorizer.transform([query])  # Transform the query into TF-IDF representation

# Calculate cosine similarity between the query and all documents
cosine_similarities = cosine_similarity(query_tfidf, tfidf_matrix).flatten()

# Create a DataFrame to store document information and similarity scores
result_df = pd.DataFrame({
    'courseName': row['courseName'],
    'universityName': df['universityName'],
    'description': df['description'],
    'url': df['url'],
    'similarity_score': cosine_similarities
})

# Sort the documents based on similarity scores in descending order
result_df = result_df.sort_values(by='similarity_score', ascending=False)



In [16]:
# Format similarity scores to have five decimal places
result_df['similarity_score'] = result_df['similarity_score'].apply(lambda x: round(x, 5))

# Print the sorted DataFrame
result_df.head()

,courseName,universityName,description,url,similarity_score
2962,Materials Engineering,Cardiff Metropolitan University,The aim of the MSc Entrepreneurship and Innova...,https://www.findamasters.com/masters-degrees/c...,0.26415
4764,Materials Engineering,Cardiff University,Why study this courseThis course aims to equip...,https://www.findamasters.com/masters-degrees/c...,0.26325
508,Materials Engineering,University College London,Register your interest in graduate study at UC...,https://www.findamasters.com/masters-degrees/c...,0.24586
1025,Materials Engineering,Vytautas Magnus University,The aim of the Applied Informatics Master stud...,https://www.findamasters.com/masters-degrees/c...,0.24046
2226,Materials Engineering,Coventry University,The MSc in Cyber Security aims to provide you ...,https://www.findamasters.com/masters-degrees/c...,0.23876


It seems good that the same courses in different universitiess have high similarity.

# 7. Algorithmic Question

1. Implement a code to solve the above mentioned problem.

2. What is the time complexity (the Big O notation) of your solution? Please provide a detailed explanation of how you calculated the time complexity.

3. Ask ChatGPT or any other LLM chatbot tool to check your code's time complexity (the Big O notation). Compare your answer to theirs. Do you believe this is correct? If the two differ, which one is right? (why?)

4. What do you think of the optimality of your code? Do you believe it is optimal? Can you improve? Please elaborate on your response.

## 7.1 Implementing a Solution

Our solution is based on binary search. Basically our solution works as follows:
1. add up Flexibility (maximal working hours - minimal working hours) and store it in an array (this array is by construction sorted)
2. compute remaining working hours (i.e. hours not already consumed by the minimal working hours)
3. search (by binary search) for the first index (critical day) in the Flexibility array, where Flexibility is greater than or equal to the remaining working hours.
4. Print upper working hours until the critical day, print the hours for the critical day, print the minimal working hours for the remaining days. 

Let us first define an appropriate version of binary search.

In [ ]:
def binary_search_geq(arr, constant, low, high):
    """
    Takes a sorted array (arr) as well as the indexes low and high.
    Assumes low <= high.
    Searches for the first (index,value)-pair where value is greater than or equal to a constant.
    Returns the modified tuple (index, value-constant)
    This method is just a properly adjusted version of the binary_search-algorithm.
    """
    # if low <= high:
    mid = (low+high)//2
    # print(mid)
    if arr[mid] >= constant:
        if mid == 0 : # is it the first element? 
            return mid #, arr[mid] - constant # We've found the searched for tuple!
        elif arr[mid - 1] < constant: # is the value at index mid-1 still <= 0 ?
            return mid # , arr[mid] - constant # We've found the searched for tuple!
        else:
            return binary_search_geq(arr, constant, low, mid - 1) # search goes on in the left part of the array.
    else: 
        return binary_search_geq(arr, constant, mid + 1, high) # search goes on in the right part of the array

It follows the code to run the problem.

In [ ]:
# d stands for "total_days
def provide_report():
    d, sumHours = map(int, input().split())
    remHours = sumHours # remaining hours
    total_flexibility = 0
    lower_bounds = []
    upper_bounds = []
    cumFlex = [] # cumulated flexibility


    for _ in range(d):
        lower, upper = map( int, input().split() )
        lower_bounds.append(lower)
        upper_bounds.append(upper)
        remHours -= lower
        total_flexibility += upper-lower
        cumFlex.append(total_flexibility) # this gives a sorted array

    if remHours < 0 or remHours > total_flexibility:
        print('NO')
    else:
        print('YES')
        crit_day = binary_search_geq(cumFlex, remHours, 0 , d-1)
        crit_hours = lower_bounds[crit_day]
        if crit_day > 0:
            crit_hours += remHours - cumFlex[crit_day - 1] # greater than or equal to zero by above binary search.
        else:
            crit_hours += remHours

        working_more_than_minimal = upper_bounds[:crit_day]
        working_more_than_minimal.append(crit_hours)

        # report = working_more_than_minimal + lower_bounds[crit_day+1:]
        print(*working_more_than_minimal, *lower_bounds[crit_day+1:] )


Examples

In [ ]:
provide_report()

YES
10 0


## 7.2 Time Complexity
We consider the function provide_report(). The input needs O(d) time, where d is the total number of days and the print-statement at the end of provide_report() takes also O(d). However this should not be the focus of this consideration. Let's consider the part inbetween, after the last input-pair is given.


The only critical step is calling the function "binary_search_geq()". All the other steps clearly just need constant time. However binary search runs in O(log(d)) by lecture where d is the total number of days. And this is optimal for this searching problem.

## 7.3 chatGPT
Chat-GPT agrees on our analysis of the time complexity.

## 7.4 Optimality
**Question 1: Can we start providing a report before receiving the last input-pair?**

In the worst case the answer is "no". The last input-pair can change our complete solution. Consider for example the input
2 2 

0 1 

0 1 

which needs 1 1 as a solution. And the input

2 1 

0 1 

2 2 

needs 0 2 as a solution.



**Question 2: Can the problem be reduced to a simpler problem?**

Possible, but we haven't found a better approach and also believe there is none.

**Resumé**

Hence we believe in the optimality of our code.